In [74]:
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import json
from census import Census
from config import (census_key, gkey)
import gmaps

# Census API Key
c = Census(census_key)

In [75]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E", "B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_008E", "B03001_003E"
                         ), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B02001_002E": "Caucasian",
                                      "B02001_003E": "African American",
                                      "B02001_004E": "American Indian",
                                      "B02001_005E": "Asian",
                                      "B02001_006E": "Native Hawaiian",
                                      "B02001_008E": "Two or more races",
                                      "B03001_003E": "Hispanic",
                                      "zip code tabulation area": "Zipcode"})

In [76]:
census_pd.sample(5)

,Population,Poverty Count,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode
15840,12631.0,935.0,11978.0,218.0,70.0,85.0,0.0,204.0,573.0,26,48173
29560,2411.0,1253.0,17.0,4.0,2322.0,5.0,0.0,63.0,33.0,35,87328
13406,2157.0,565.0,2157.0,0.0,0.0,0.0,0.0,0.0,0.0,21,41553
12423,2667.0,290.0,2088.0,458.0,0.0,14.0,0.0,107.0,0.0,47,38356
1122,1034.0,161.0,997.0,0.0,26.0,3.0,0.0,8.0,61.0,23,04292


In [77]:
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"]/ census_pd["Population"]

In [78]:
census_pd.sample(5)

,Population,Poverty Count,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode,Poverty Rate
16907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,50319,NaN
345,4870.0,121.0,4384.0,358.0,0.0,28.0,6.0,44.0,133.0,25,01612,2.484600
29357,4447.0,519.0,3715.0,40.0,391.0,0.0,0.0,199.0,1763.0,04,85936,11.670789
21342,15121.0,5423.0,5306.0,9206.0,44.0,273.0,0.0,243.0,77.0,17,62206,35.864030
14419,21043.0,1749.0,20517.0,26.0,0.0,63.0,0.0,409.0,246.0,39,44654,8.311553


In [79]:
census_pd.columns = census_pd.columns.str.replace(' ', '_').str.lower()
census_pd.tail(5)

,population,poverty_count,caucasian,african_american,american_indian,asian,native_hawaiian,two_or_more_races,hispanic,state,zipcode,poverty_rate
33115,15.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99923,0.000000
33116,927.0,172.0,446.0,3.0,366.0,0.0,7.0,105.0,26.0,02,99925,18.554477
33117,1635.0,235.0,239.0,0.0,1210.0,22.0,0.0,147.0,116.0,02,99926,14.373089
33118,38.0,28.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99927,73.684211
33119,2484.0,192.0,1670.0,2.0,496.0,69.0,9.0,208.0,121.0,02,99929,7.729469


### CSV uploaded

In [5]:
value_df = pd.read_csv("Payment_and_Value_of_Care-Hospital.csv", low_memory=False)
value_df

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,TRAVIS,(512) 654-2100,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,Not Available,Not Available,Not Available,5.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,5.0,04/01/2016,03/31/2019
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,19.0,07/01/2016,06/30/2019
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_HF,Payment for heart failure patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,19.0,07/01/2016,06/30/2019
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_PN,Payment for pneumonia patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,19.0,07/01/2016,06/30/2019


In [6]:
value_df.columns = value_df.columns.str.replace(' ', '_').str.lower()
value_df.head()

,facility_id,facility_name,address,city,state,zip_code,county_name,phone_number,payment_measure_id,payment_measure_name,...,payment,lower_estimate,higher_estimate,payment_footnote,value_of_care_display_id,value_of_care_display_name,value_of_care_category,value_of_care_footnote,start_date,end_date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019


In [7]:
for column in value_df.columns:
    print(column)

facility_id
facility_name
address
city
state
zip_code
county_name
phone_number
payment_measure_id
payment_measure_name
payment_category
denominator
payment
lower_estimate
higher_estimate
payment_footnote
value_of_care_display_id
value_of_care_display_name
value_of_care_category
value_of_care_footnote
start_date
end_date


In [8]:
updated_df=value_df.drop(['county_name', 'phone_number', 'payment_footnote', 'value_of_care_footnote'], axis=1).reset_index(drop=True)
updated_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_id,payment_measure_name,payment_category,denominator,payment,lower_estimate,higher_estimate,value_of_care_display_id,value_of_care_display_name,value_of_care_category,start_date,end_date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934","$23,490","$26,429",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854","$17,061","$18,676",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216","$19,186","$21,271",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216","$21,191","$23,313",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742","$22,016","$27,967",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Not Available,Not Available,Not Available,Not Available,Not Available,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,04/01/2016,03/31/2019
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,PAYM_30_AMI,Payment for heart attack patients,Not Available,Not Available,Not Available,Not Available,Not Available,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,07/01/2016,06/30/2019
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,PAYM_30_HF,Payment for heart failure patients,Not Available,Not Available,Not Available,Not Available,Not Available,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,07/01/2016,06/30/2019
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,PAYM_30_PN,Payment for pneumonia patients,Not Available,Not Available,Not Available,Not Available,Not Available,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,07/01/2016,06/30/2019


In [9]:
no_nans_df = updated_df[updated_df.payment != 'Not Available']
no_nans_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_id,payment_measure_name,payment_category,denominator,payment,lower_estimate,higher_estimate,value_of_care_display_id,value_of_care_display_name,value_of_care_category,start_date,end_date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934","$23,490","$26,429",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854","$17,061","$18,676",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216","$19,186","$21,271",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216","$21,191","$23,313",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742","$22,016","$27,967",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,143,"$20,221","$18,213","$22,129",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,53,"$27,138","$23,811","$30,881",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,PAYM_30_HF,Payment for heart failure patients,Greater Than the National Average Payment,155,"$21,647","$19,686","$23,716",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,07/01/2016,06/30/2019
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,205,"$19,409","$17,835","$21,020",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,07/01/2016,06/30/2019


In [10]:
no_nans_df.state.value_counts()

CA    912
TX    891
FL    625
IL    533
PA    500
NY    488
OH    483
MI    375
IN    338
NC    333
GA    333
WI    329
MO    295
VA    271
IA    259
TN    258
KY    248
KS    244
LA    241
OK    239
NJ    237
WA    230
AL    226
MA    214
MN    208
AZ    203
CO    200
MS    198
AR    189
SC    170
OR    170
MD    167
NE    165
WV    121
UT    110
ME    108
CT     99
NV     98
NM     93
MT     88
ID     88
NH     87
SD     82
ND     62
PR     59
WY     57
VT     46
HI     43
AK     36
RI     35
DE     24
DC     21
VI      6
GU      3
MP      3
Name: state, dtype: int64

In [11]:
no_nans_df.payment_measure_id.unique()

array(['PAYM_30_AMI', 'PAYM_30_HF', 'PAYM_30_PN', 'PAYM_90_HIP_KNEE'],
      dtype=object)

In [12]:
no_nans_df.payment_measure_name.unique()

array(['Payment for heart attack patients',
       'Payment for heart failure patients',
       'Payment for pneumonia patients',
       'Payment for hip/knee replacement patients'], dtype=object)

In [13]:
no_nans_df.payment_category.unique()

array(['No Different Than the National Average Payment',
       'Greater Than the National Average Payment',
       'Less Than the National Average Payment'], dtype=object)

In [21]:
zipcodes=no_nans_df.zip_code.unique()

In [14]:
no_nans_df['payment'].apply(type).value_counts()

<class 'str'>    12141
Name: payment, dtype: int64

In [15]:
# no_nans_df['payment'] = no_nans_df['payment'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# no_nans_df.lower_estimate = no_nans_df.lower_estimate.replace({'\$': '', ',': ''}, regex=True).astype(float)
# no_nans_df.higher_estimate = no_nans_df.higher_estimate.replace({'\$': '', ',': ''}, regex=True).astype(float)
# no_nans_df.denominator = no_nans_df.higher_estimate.replace({'\$': '', ',': ''}, regex=True).astype(float)

In [17]:
cols = ['denominator', 'payment', 'lower_estimate', 'higher_estimate']
no_nans_df[cols] = no_nans_df[cols].applymap(lambda x: x.replace('$', '').replace(',', '')).astype(float)
no_nans_df.sample(5)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,facility_id,facility_name,address,city,state,zip_code,payment_measure_id,payment_measure_name,payment_category,denominator,payment,lower_estimate,higher_estimate,value_of_care_display_id,value_of_care_display_name,value_of_care_category,start_date,end_date
15849,450271,WISE REGIONAL HEALTH SYSTEM,609 MEDICAL CENTER DRIVE,DECATUR,TX,76234,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,187.0,19067.0,17509.0,20804.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,07/01/2016,06/30/2019
1838,50523,SUTTER DELTA MEDICAL CENTER,3901 LONE TREE WAY,ANTIOCH,CA,94509,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,314.0,18246.0,17050.0,19472.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
16104,450647,MEDICAL CITY DALLAS HOSPITAL,7777 FOREST LANE,DALLAS,TX,75230,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,149.0,27167.0,24578.0,29927.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,07/01/2016,06/30/2019
3877,110121,GRADY GENERAL HOSPITAL,"1155 5TH STREET, SE",CAIRO,GA,39828,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,47.0,15750.0,13769.0,17910.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,07/01/2016,06/30/2019
3190,100126,PALMS OF PASADENA HOSPITAL,1501 PASADENA AVE S,SAINT PETERSBURG,FL,33707,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,264.0,19576.0,18147.0,20926.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,07/01/2016,06/30/2019


In [19]:
no_nans_df.payment.mean()

20004.158718392224

In [20]:
no_nans_df.groupby('state').mean()

,facility_id,zip_code,denominator,payment,lower_estimate,higher_estimate
state,,,,,,
AK,20301.944444,99641.388889,165.638889,17643.861111,16018.027778,19399.416667
AL,10092.584071,35911.827434,262.663717,19343.256637,17702.420354,21097.486726
AR,40382.756614,72294.619048,249.132275,19722.380952,17993.312169,21553.873016
AZ,30167.349754,85512.906404,321.802956,20006.866995,18478.364532,21628.655172
CA,50387.296053,93204.076754,280.947368,20693.654605,19013.044956,22503.525219
CO,60306.215000,80673.815000,204.370000,19606.700000,17802.495000,21547.790000
CT,70019.343434,6415.717172,458.414141,20828.343434,19459.828283,22283.030303
DC,90005.380952,20026.142857,346.047619,21129.523810,19570.619048,22822.142857
DE,80005.000000,19886.333333,675.291667,20432.791667,19208.125000,21737.916667
